# Tabular Playground Series - Jul 2021
まずは、データの中身を確認しました。
target_carbon_monoxide、target_benzene、target_nitrogen_oxidesの3種類を予測する必要があるようです。ベンゼンが入っていることから、自動車の排気ガスによる大気汚染の予測ではないかと思われます。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

## Overview

In [ ]:
train

In [ ]:
train.info()

In [ ]:
train.describe().T

In [ ]:
test

In [ ]:
test.info()

In [ ]:
sub

## Correlation

散布図を見ると、センサー1～5は、ベンゼンとは強い相関がありますが、一酸化炭素や窒素酸化物との相関は外れ値が目立ちます。

### Correlation coefficient


In [ ]:
plt.figure(figsize=(9,9))
sns.heatmap(train.drop('date_time',axis = 1).corr(), vmax=1, square=True,annot=True,cmap='RdBu')
plt.show()

### Distribution chart

In [ ]:
sns.pairplot(train.drop('date_time',axis = 1))

相関を見ると  
　・target_benzeneとsensor2は、強い正の相関  
　・target_carbon_monoxideとtarget_benzeneは、外れ値を対象外にすれば強い正の相関   
　・target_nitrogen_oxidesとtarget_carbon_monoxideは、やや強い正の相関  
があることがわかります。

これらを大きな図で見てみます。

### Relationship between benzene and sensor_2
target_benzeneとsensor_2は、普通に式が書けそうなぐらい正の強い相関がありますね。

In [ ]:
sns.jointplot('target_benzene', 'sensor_2', data=train)

### Relationship between carbon monoxide and benzene
target_carbon_monoxideとtarget_benzeneは強めの正の相関ですが、外れ値が目立ちます。

In [ ]:
sns.jointplot('target_carbon_monoxide', 'target_benzene', data=train)

### Relationship between carbon monoxide and nitrogen oxides
target_nitrogen_oxidesとtarget_carbon_monoxideは、まぁまぁな正の相関があります。外れ値は見当たりません。

In [ ]:
sns.jointplot('target_nitrogen_oxides', 'target_carbon_monoxide', data=train)

## Predict and check SHAP with LightGBM
SHAPで特徴量の重要度を確認したいので、いったん、PyCaretを使ってLightGBMで予測してSHAPを表示します。

In [ ]:
!pip install pycaret==2.3.1

In [ ]:
!pip install shap

In [ ]:
from pycaret.regression import setup, create_model, tune_model, finalize_model, predict_model, interpret_model
import shap

### Carbon monoxide

In [ ]:
train1 = train.drop(['date_time','target_benzene','target_nitrogen_oxides'], axis=1)
train1.head()

In [ ]:
reg1 = setup(data=train1, target='target_carbon_monoxide', session_id=1)

In [ ]:
selected1 = create_model("lightgbm")
tuned1 = tune_model(selected1)

In [ ]:
final1 = finalize_model(tuned1)
pred1 = predict_model(final1, data=test)
interpret_model(final1)

### Benzene

In [ ]:
train2 = train.drop(['date_time','target_carbon_monoxide','target_nitrogen_oxides'], axis=1)
train2.head()

In [ ]:
reg2 = setup(data=train2, target='target_benzene', session_id=2)

In [ ]:
selected2 = create_model("lightgbm")
tuned2 = tune_model(selected2)

In [ ]:
final2 = finalize_model(tuned2)
pred2 = predict_model(final2, data=test)
interpret_model(final2)

## Nitrogen oxides

In [ ]:
train3 = train.drop(['date_time','target_carbon_monoxide','target_benzene'], axis=1)
train3.head()

In [ ]:
reg3 = setup(data=train3, target='target_nitrogen_oxides', session_id=3)

In [ ]:
selected3 = create_model("lightgbm")
tuned3 = tune_model(selected3)

In [ ]:
final3 = finalize_model(tuned3)
pred3 = predict_model(final3, data=test)
interpret_model(final3)

Carbon monoxide、Benzeneはsensor_2、Nitrogen oxidesはsensor_5が予測する際に重要度が高いようです。  
これらをサブミットしてみます。

In [ ]:
sub.target_carbon_monoxide = pred1.Label
sub.target_benzene = pred2.Label
sub.target_nitrogen_oxides = pred3.Label
sub.to_csv('lightgbm_submission.csv',index=False)
sub

スコア : **0.30025** 